In [1]:
test_data_raw = """1,0,1~1,2,1
0,0,2~2,0,2
0,2,3~2,2,3
0,0,4~0,2,4
2,0,5~2,2,5
0,1,6~2,1,6
1,1,8~1,1,9"""


def parse_data(data_raw):
    return [
        [tuple(map(int, coord.split(","))) for coord in line.split("~")]
        for line in data_raw.splitlines()
    ]


test_data = parse_data(test_data_raw)
test_data

[[(1, 0, 1), (1, 2, 1)],
 [(0, 0, 2), (2, 0, 2)],
 [(0, 2, 3), (2, 2, 3)],
 [(0, 0, 4), (0, 2, 4)],
 [(2, 0, 5), (2, 2, 5)],
 [(0, 1, 6), (2, 1, 6)],
 [(1, 1, 8), (1, 1, 9)]]

In [3]:
def get_space_shape(bricks):
    shape = [0, 0, 0]
    for start, end in bricks:
        for axis in [0, 1, 2]:
            shape[axis] = max(shape[axis], start[axis] + 1, end[axis] + 1)
    return tuple(shape)


get_space_shape(test_data)

(3, 3, 10)

In [91]:
import numpy as np


def get_brick_coords(start, end):
    for axis in range(3):
        for value in range(start[axis], end[axis] + 1):
            new_coord = list(start)
            new_coord[axis] = value
            yield tuple(new_coord)


def id2letter(id):
    return chr(ord("A") + id)


def fall_bricks(bricks, space):
    new_bricks = []
    for brick_id, brick in enumerate(bricks):
        new_brick = (list(brick[0]), list(brick[1]))
        while min(new_brick[0][2], new_brick[1][2]) >= 0 and all(map(lambda coord: space[coord] < 0, get_brick_coords(*new_brick))):
            new_brick[0][2] -= 1
            new_brick[1][2] -= 1
        new_brick[0][2] += 1
        new_brick[1][2] += 1
        for coords in get_brick_coords(*new_brick):
            space[coords] = brick_id
        new_brick = (tuple(new_brick[0]), tuple(new_brick[1]))
        new_bricks.append(new_brick)
    return new_bricks


def get_supports(bricks, space):
    supports = []
    for brick_id, brick in enumerate(bricks):
        current_supports = set()
        for x, y, z in get_brick_coords(*brick):
            zz = z + 1
            if zz < space.shape[2] and space[x, y, zz] > 0 and space[x, y, zz] != brick_id:
                current_supports.add(space[x, y, zz])
        supports.append(current_supports)
    return supports


def part1(data):
    shape = get_space_shape(data)
    space = np.full(shape, -1)

    new_data = fall_bricks(data, space)
    supports = get_supports(new_data, space)

    # print(*[
    #     f"{id2letter(brick_id + 1)}: {set(map(id2letter, current_support))}"
    #     for brick_id, current_support in enumerate(supports)
    # ], sep="\n")

    result = 0
    for brick_id, support in enumerate(supports):
        other_supports = set.union(*(supports[:brick_id] + supports[brick_id + 1:]))
        # print(support, other_supports, support - other_supports)
        if len(support - other_supports) == 0:
            result += 1

    return result


part1(test_data)

5

In [92]:
with open("input.txt") as f:
    data = parse_data(f.read())

data[:5]

[[(6, 9, 184), (8, 9, 184)],
 [(7, 0, 46), (9, 0, 46)],
 [(4, 2, 78), (4, 5, 78)],
 [(0, 4, 182), (0, 6, 182)],
 [(7, 4, 249), (7, 6, 249)]]

In [93]:
part1(data)

522